# Matrix Multiplication

In [1]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening user mode device driver
2024-01-29 16:41:46.800 | INFO     | SiliconDriver   - Detected 1 PCI device : {0}
2024-01-29 16:41:46.810 | WARNING  | SiliconDriver   - init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2024-01-29 16:41:46.810 | WARNING  | SiliconDriver   - Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2024-01-29 16:41:46.812 | WARNING  | SiliconDriver   - bind_area_memory_nodeset(): Unable to determine TT Device to NumaNode mapping for physical_device_id: 0. Skipping membind.
---- ttSiliconDevice::init_hugepage: bind_area_to_memory_nodeset() failed (physical_device_id: 0 ch: 0). Hugepage allocation is not on NumaNode matching TT Device. Side-Effect is decreased Device->Host perf (Issue #893).
                  Metal | INFO     | AI CLK for device 0 is:

## Enable program cache

Enabling the program cache will speed up the execution of operations that run repeatedly

In [2]:
ttnn.enable_program_cache()

                     Op | INFO     | Program Cache: enabled.


# Configuration

In [3]:
m = 1024
k = 1024
n = 1024

## Initialize tensors a and b with random values using torch

In [4]:
torch_a = torch.randn((m, k), dtype=torch.bfloat16)
torch_b = torch.randn((k, n), dtype=torch.bfloat16)

In [5]:
a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device)
b = ttnn.to_device(b, device)

                     Op | INFO     | Finished Operation ttnn.from_torch                                    in          225178 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           35309 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_device                                     in          949984 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_device                                     in          838825 nanoseconds


## Matrix multiply tensor a and b
The operation will run longer the first time because the kernels need to get compiled

In [6]:
output = a @ b

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           38790 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           20710 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Tilize                               in       484012355 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Tilize                               in       484148844 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Tilize                               in         1534322 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Tilize                               in         1584541 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Matmul                               in       562087924 nanoseconds
                     Op | INFO    

Re-running the operation shows significant speed up by utilizing program caching

In [7]:
output = a @ b

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           31730 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           23860 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Tilize                               in         1545421 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Tilize                               in         1587351 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Tilize                               in         1532711 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Tilize                               in         1546921 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Matmul                               in         1185983 nanoseconds
                     Op | INFO    

## Inspect the layout of matrix multiplication output

In [8]:
print(output.layout)

Layout.TILE


As can be seen, matrix multiplication produces outputs in a tile layout. That is because it's much more efficient to use this layout for computing matrix multiplications on TensTorrent accelerators compared to a row-major layout.

And this is aslo why the logs show 2 tilize operations, as the inputs get automatically convered to the tile layout if they are in a row-major layout.

Learn more about tile layout here: TODO

## Inspect the result of the matrix multiplication

To inspect the results we will first convert to row-major layout.

In [9]:
output = ttnn.to_layout(output, ttnn.ROW_MAJOR_LAYOUT)

print("Printing ttnn tensor")
print(f"shape: {output.shape}")
print(f"chunk of a tensor:\n{output[:1, :32]}")

                     Op | INFO     | Printing ttnn tensor
shape: ttnn.Shape([1024, 1024])
Finished Program   tt::tt_metal::Untilize                             in       456451455 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Untilize                             in       456574574 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_device                                   in         2053188 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_torch                                      in          533017 nanoseconds
chunk of a tensor:
Tensor([ [34.25, 9.625, 11.3125, 0.964844, 1.45312, -26.875, 23.125, -1.39062, -20.375, 33, 5.8125, 10.6875, -18.625, 14.5, -42.75, -18.375, 27.75, 44.25, -27.25, -20.5, 43.5, -5.75, -46.75, -45.75, 43.75, 33, -16.125, 39.25, 11.6875, 9.4375, -39.75, -6.5625]], dtype=bfloat16 )

                     Op | INFO     | Finished Operation torch.Tensor.__getitem__                

## Tilize tensors before running matrix multiplication
Inputs can be tilized before running matrix multiplication. 

And there are times when it's highly recommended to do that. For example, during inference, pre-tilizing weights gets rid of unnecessarily tilizing the weights every time a matmul is running

In [10]:
a = ttnn.to_layout(a, ttnn.TILE_LAYOUT)
b = ttnn.to_layout(b, ttnn.TILE_LAYOUT)

                     Op | INFO     | Finished Program   tt::tt_metal::Tilize                               in         1827330 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Tilize                               in         1912949 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Tilize                               in         1536851 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Tilize                               in         1558681 nanoseconds


Running matrix multiplication with tilized inputs shows that only matrix multiplication operation is invoked without any additional pre-processing operations

In [11]:
output = a @ b

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           35010 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           23800 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Matmul                               in         1198393 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Matmul                               in         1225833 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           25070 nanoseconds


## Matrix multiply tensor a and b by using more performant config
By default, matrix multiplication might not be as effecient as it could be. To speed it up further, the user can specify how many cores they want matrix multiplication to use. This can speed up the operation significantly.

In [12]:
a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device, memory_config=ttnn.L1_MEMORY_CONFIG)
b = ttnn.to_device(b, device, memory_config=ttnn.L1_MEMORY_CONFIG)

a = ttnn.to_layout(a, ttnn.TILE_LAYOUT)
b = ttnn.to_layout(b, ttnn.TILE_LAYOUT)

                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           49290 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           34560 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_device                                     in          832296 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_device                                     in          950575 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Tilize                               in       406539472 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Tilize                               in       406609712 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Tilize                               in          448407 nanoseconds
                     Op | INFO    

Run once to compile the kernels

In [13]:
output = ttnn.matmul(a, b, memory_config=ttnn.L1_MEMORY_CONFIG, core_grid=(8, 8))

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           63309 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           26090 nanoseconds
                     Op | INFO     | Finished Program   tt::operations::primary::Matmul                    in       601022529 nanoseconds
                     Op | INFO     | Finished Operation tt::operations::primary::Matmul                    in       601438087 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in          100719 nanoseconds


Enjoy a massive speed up on the subsequent runs

In [14]:
output = ttnn.matmul(a, b, memory_config=ttnn.L1_MEMORY_CONFIG, core_grid=(8, 8))

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           32349 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           21900 nanoseconds
                     Op | INFO     | Finished Program   tt::operations::primary::Matmul                    in          129279 nanoseconds
                     Op | INFO     | Finished Operation tt::operations::primary::Matmul                    in          177849 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           24300 nanoseconds


## Close the device

In [15]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
